In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer


In [2]:
base_path = os.path.join('', 'data')
train_path = os.path.join(base_path, 'Train.csv')
test_path = os.path.join(base_path, 'Test.csv')

In [3]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [5]:
# shape of the train and test datasets
print(f'train_df shape {train_df.shape} and test_df shape: {test_df.shape}')

train_df shape (8071, 80) and test_df shape: (2783, 79)


In [ ]:

# removing variables with more than 45% missing values
drop_cols = [i for i in train_df.columns if train_df[i].isnull().sum() / len(train_df) > 0.45]
print(f'the number of columns to drop is {len(np.array(drop_cols))}')

the number of columns to drop is 36
